<a href="https://colab.research.google.com/github/achmfirmansyah/sweet_project/blob/master/ICST2020/03_BUildup_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install rasterio
import rasterio
!pip install geopandas
import geopandas as gpd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
from sklearn.model_selection import train_test_split
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

     |████████████████████████████████| 18.2MB 1.3MB/s 
     |████████████████████████████████| 962kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 197kB/s 
     |████████████████████████████████| 14.7MB 299kB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
lokus=['jabodetabek','mebidangro','maminasata','kedungsepur']
confussion_matrix_list=[]
classification_report_list=[]

In [ ]:
#Create model
for lokasi in lokus:
  print(lokasi)
  dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  dataset_bands=pd.DataFrame()
  for i in dataset.indexes:
    temp=dataset.read(i)
    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
    dataset_bands=temp.join(dataset_bands)
  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
                                                    dataset[['U_class']], test_size = 0.20)
  xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
                               learning_rate=0.05, max_depth= 4, 
                             min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
                             seed= 123, silent=1, subsample= 0.8)
  xgclassifier.fit(X_train.values,y_train.values)
  y_pred=xgclassifier.predict(X_test.values)
  #confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  classification_report_list.append(classification_report(y_test, y_pred))
  dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  class_2014_=xgclassifier.predict(dataset.values)
  confussion_matrix_list.append(confusion_matrix(dataset_bands[['U_class']],class_2014_))
  temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  temp = temp_.read(1)
  
  array_class_2014=class_2014_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  
  profile = temp_.profile
  profile.update(
        dtype=array_class_2014.dtype,
        count=1,
        compress='lzw')
  with rasterio.open(
    '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2014_'+lokasi+'.tif','w',**profile) as dst2:
    dst2.write(array_class_2014,1)
    dst2.close()

jabodetabek


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mebidangro
maminasata
kedungsepur


In [ ]:
confussion_matrix_list[3]

array([[6239065, 6616121],
       [  39957, 1098329]])

In [ ]:
#Create model
for lokasi in lokus:
  print(lokasi)
  dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  dataset_bands=pd.DataFrame()
  for i in dataset.indexes:
    temp=dataset.read(i)
    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
    dataset_bands=temp.join(dataset_bands)
  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
                                                    dataset[['U_class']], test_size = 0.20)
  xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
                               learning_rate=0.05, max_depth= 4, 
                             min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
                             seed= 123, silent=1, subsample= 0.8)
  xgclassifier.fit(X_train.values,y_train.values)
  y_pred=xgclassifier.predict(X_test.values)
  confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2019:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2019_=xgclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2019=class_2019_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2019.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2019_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2019,1)
  #  dst2.close()

jabodetabek


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mebidangro
maminasata
kedungsepur


In [ ]:
#Create model
for lokasi in lokus:
  #print(lokasi)
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  #dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  #X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
  #                                                  dataset[['U_class']], test_size = 0.20)
  #xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
  #                             learning_rate=0.05, max_depth= 4, 
  #                           min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
  #                           seed= 123, silent=1, subsample= 0.8)
  #xgclassifier.fit(X_train.values,y_train.values)
  #y_pred=xgclassifier.predict(X_test.values)

  #classification_2015:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2015_=xgclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2015=class_2015_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2015.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2015_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2015,1)
  #  dst2.close()

jabodetabek


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mebidangro
maminasata
kedungsepur


In [ ]:
lokus=['gerbangkertasusila']
#Create model
for lokasi in lokus:
  print(lokasi)
  dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  dataset_bands=pd.DataFrame()
  for i in dataset.indexes:
    temp=dataset.read(i)
    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
    dataset_bands=temp.join(dataset_bands)
  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
                                                    dataset[['U_class']], test_size = 0.20)
  xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
                               learning_rate=0.05, max_depth= 5, 
                             min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
                             seed= 123, silent=1, subsample= 0.8)
  xgclassifier.fit(X_train.values,y_train.values)
  #y_pred=xgclassifier.predict(X_test.values)
  y_pred=xgclassifier.predict(dataset_bands[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']].values)
  confussion_matrix_list.append(confusion_matrix(dataset_bands[['U_class']],y_pred))
  #confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  #classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2015:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2015_=xgclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2015=class_2015_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2015.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2015_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2015,1)
  #  dst2.close()
#Create model
#for lokasi in lokus:
 # print(lokasi)
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
   # temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
   # dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
   #                         5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  #dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  #X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
  #                                                  dataset[['U_class']], test_size = 0.20)
  #xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
  #                             learning_rate=0.05, max_depth= 5, 
  #                           min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
  #                           seed= 123, silent=1, subsample= 0.8)
  #xgclassifier.fit(X_train.values,y_train.values)
  #y_pred=xgclassifier.predict(X_test.values)
  #confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  #classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2019:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2019_=xgclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2019=class_2019_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2019.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2019_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2019,1)
  #  dst2.close()

#Create model
#for lokasi in lokus:
  #print(lokasi)
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  #dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  #X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
  #                                                  dataset[['U_class']], test_size = 0.20)
  #xgclassifier = XGBClassifier(random_state=123,colsample_bytree= 0.7, 
  #                             learning_rate=0.05, max_depth= 5, 
  #                           min_child_weight=11, n_estimators= 500, nthread= 4, objective= 'binary:logistic', 
  #                           seed= 123, silent=1, subsample= 0.8)
  #xgclassifier.fit(X_train.values,y_train.values)
  #y_pred=xgclassifier.predict(X_test.values)
  #confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  #classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2019:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2014_=xgclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2014=class_2014_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2014.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2014_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2014,1)
  #  dst2.close()

gerbangkertasusila


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
confussion_matrix_list[4]

array([[8910520,  891343],
       [  55632,  671994]])

In [ ]:
lokus=['bandungraya']
from sklearn.ensemble import RandomForestClassifier
#Create model
for lokasi in lokus:
  print(lokasi)
  dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
  dataset_bands=pd.DataFrame()
  for i in dataset.indexes:
    temp=dataset.read(i)
    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
    dataset_bands=temp.join(dataset_bands)
  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
                                                    dataset[['U_class']], test_size = 0.20)
  rfclassifier = RandomForestClassifier(random_state=123,max_depth=6,n_estimators=500,criterion='entropy')
  rfclassifier.fit(X_train.values,y_train.values)
  y_pred=rfclassifier.predict(dataset_bands[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']].values)
  confussion_matrix_list.append(confusion_matrix(dataset_bands['U_class'],y_pred))
  #y_pred=rfclassifier.predict(X_test.values)

  #classification_2014:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  # dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  ##dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2015_=rfclassifier.predict(dataset.values)
  #dataset_bands['U_class']
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2015_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2015=class_2015_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2015.dtype,
   #     count=1,
   #     compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2015_'+lokasi+'_2.tif','w',**profile) as dst2:
   # dst2.write(array_class_2015,1)
   # dst2.close()
#Create model
#for lokasi in lokus:
 # print(lokasi)
 # dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
 # dataset_bands=pd.DataFrame()
 # for i in dataset.indexes:
 #   temp=dataset.read(i)
 #   temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
 #   dataset_bands=temp.join(dataset_bands)
 # dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
  #dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
  #X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
  #                                                  dataset[['U_class']], test_size = 0.20)
  #rfclassifier = RandomForestClassifier(random_state=123,max_depth=6,n_estimators=500,criterion='entropy')
  #rfclassifier.fit(X_train.values,y_train.values)
  #y_pred=rfclassifier.predict(X_test.values)
  #confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
  #classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2019:
  #dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #dataset_bands=pd.DataFrame()
  #for i in dataset.indexes:
  #  temp=dataset.read(i)
  #  temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
  #  dataset_bands=temp.join(dataset_bands)
  #dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
  #                          5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
  #dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
  #dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
  #  0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
  #dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
  #class_2019_=rfclassifier.predict(dataset.values)
  #temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2019_'+lokasi+'.tif')
  #temp = temp_.read(1)
  #array_class_2019=class_2019_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
  #profile = temp_.profile
  #profile.update(
  #      dtype=array_class_2019.dtype,
  #      count=1,
  #      compress='lzw')
  #with rasterio.open(
  #  '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2019_'+lokasi+'.tif','w',**profile) as dst2:
  #  dst2.write(array_class_2019,1)
  #  dst2.close()

#Create model
#for lokasi in lokus:
#  print(lokasi)
#  dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
#  dataset_bands=pd.DataFrame()
#  for i in dataset.indexes:
#    temp=dataset.read(i)
#    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
#    dataset_bands=temp.join(dataset_bands)
#  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
#                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
#  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
#  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
#    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
#  dataset=dataset_bands.query('U_class==0').sample(10000).append(dataset_bands.query('U_class==1').sample(10000))
#  dataset=dataset.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7','U_class']]
#  X_train, X_test, y_train, y_test = train_test_split(dataset[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDVI_landsat7','NDBI_landsat7','LSE_der_landsat7']],
#                                                    dataset[['U_class']], test_size = 0.20)
#  rfclassifier = RandomForestClassifier(random_state=123,max_depth=6,n_estimators=500,criterion='entropy')
#  rfclassifier.fit(X_train.values,y_train.values)
#  y_pred=rfclassifier.predict(X_test.values)
#  confussion_matrix_list.append(confusion_matrix(y_test, y_pred))
#  classification_report_list.append(classification_report(y_test, y_pred))

  #classification_2019:
# dataset = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
#  dataset_bands=pd.DataFrame()
#  for i in dataset.indexes:
#    temp=dataset.read(i)
#    temp=pd.DataFrame(data=np.array(temp).flatten()).rename(columns={0:i})
#    dataset_bands=temp.join(dataset_bands)
#  dataset_bands.rename(columns={1:'BU_class',2:'NDVI_landsat7',3:'NDBI_landsat7',4:'MNDWI_landsat7',
#                            5:'SAVI_landsat7',6:'LSE_landsat7',7:'rad_VIIRS'},inplace=True)
#  dataset_bands['U_class']=dataset_bands.BU_class.apply(lambda y: 1 if y>=3 else 0)
#  dataset_bands['LSE_der_landsat7']=dataset_bands.NDVI_landsat7.apply(lambda y: 0.995 if y < -0.185 else (
#    0.970 if y< 0.157 else (1.0098+0.047*np.log(y) if y<0.727 else 0.990)))
#  dataset=dataset_bands.reset_index()[['rad_VIIRS','SAVI_landsat7','MNDWI_landsat7','NDBI_landsat7','NDVI_landsat7','LSE_der_landsat7']]
#  class_2014_=rfclassifier.predict(dataset.values)
#  temp_ = rasterio.open('/content/gdrive/My Drive/Urban_monitoring/Urban_/compiled_GHSL_30_train_2014_'+lokasi+'.tif')
#  temp = temp_.read(1)
#  array_class_2014=class_2014_.reshape(temp.shape[0],temp.shape[1]).astype(np.uint8)
#  profile = temp_.profile
#  profile.update(
#        dtype=array_class_2014.dtype,
#        count=1,
#        compress='lzw')
#  with rasterio.open(
#    '/content/gdrive/My Drive/Urban_monitoring/Urban_/GHSL_/GHSL_rev/rev_class_2014_'+lokasi+'.tif','w',**profile) as dst2:
#    dst2.write(array_class_2014,1)
#    dst2.close()

bandungraya


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [ ]:
confussion_matrix_list[5]

array([[4034753, 1953292],
       [   5903,  549529]])